In [41]:
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

def detect_bab_subbab(text):
    # Pattern kasar untuk Bab & Subbab
    bab_pattern = r"(BAB\s+[IVXLCDM]+\s+.+)"
    subbab_pattern = r"(\d+(\.\d+)+\s+.+)"
    
    babs = []
    current_bab = None
    current_subbab = None

    lines = text.split("\n")
    processed_text = []
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        
        if re.match(bab_pattern, line):
            current_bab = line
        
        processed_text.append({
            "text": line,
            "bab": current_bab,
        })
    
    return processed_text

def chunk_with_metadata(processed_text, chunk_size=1000, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    
    # Gabung teks berdasarkan detected Bab/Subbab
    full_text = "\n".join([p["text"] for p in processed_text])
    chunks = text_splitter.split_text(full_text)
    
    docs = []
    for chunk in chunks:
        # Cari metadata relevan buat setiap chunk
        # Ambil dari baris pertama yang kena di chunk ini
        first_line = chunk.split("\n")[0].strip()
        found_metadata = next(
            (p for p in processed_text if p["text"] == first_line),
            {"bab": None}
        )
        docs.append(
            Document(
                page_content=chunk,
                metadata={
                    "bab": found_metadata["bab"],
                }
            )
        )
    return docs

def remove_page_numbers(text):
    # Pattern to detect page numbers (e.g., numbers at the beginning or end of lines)
    page_number_pattern = r"^\s*\d+\s*$"
    
    lines = text.split("\n")
    cleaned_lines = [line for line in lines if not re.match(page_number_pattern, line.strip())]
    
    return "\n".join(cleaned_lines)

In [44]:
with open("Data/dokumen4.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Hapus nomor halaman
text = remove_page_numbers(text)

# Step 1: Deteksi struktur
processed_text = detect_bab_subbab(text)

# Step 2: Split + Tambah Metadata
docs = chunk_with_metadata(processed_text)

In [45]:
processed_text

[{'text': 'BAB I PENDAHULUAN Pandangan secara umum',
  'bab': 'BAB I PENDAHULUAN Pandangan secara umum'},
 {'text': 'Manusia hidup berdampingan satu sama lain, Hubungan tersebut antara lain berkaitan dengan orang lain, hewan, tumbuhan, gunung, lautan, dan seluruh alam semesta. Hubungan antara diri sendiri dengan alam semesta membutuhkan keseimbangan dan sinkronitas yang cukup baik saling berdampingan. Realitasnya antara individu dengan individu lain tidak terlepas dengan adanya sebuah konflik. Konflik yang hasilkan dari gesekan individu lain memiliki bentuk dan dampak yang berbeda yang ditimbulkan oleh setiap individu. Dampak yang timbul pada umumnya akan berdampak pada diri sendiri mulai dari menyalahkan diri, menyakiti diri, sampai merasakan kehampaan yang begitu besar di timbulkan.',
  'bab': 'BAB I PENDAHULUAN Pandangan secara umum'},
 {'text': 'Permasalahan tidak hanya dari manusia lain saja, tetapi juga datang dari kerusakan lingkungan yang mengakibatkan  bencana  alam.  Kejadian

In [49]:
with open("Data/dokumen3.txt", "r", encoding="utf-8") as f:
    text = f.read()
text = remove_page_numbers(text)

processed_text = detect_bab_subbab(text)

# Step 2: Split + Tambah Metadata
docs = chunk_with_metadata(processed_text)


In [50]:
docs

[Document(metadata={'bab': 'BAB I PRINSIP-PRINSIP DAN FUNGSI KESEHATAN MENTAL'}, page_content='BAB I PRINSIP-PRINSIP DAN FUNGSI KESEHATAN MENTAL\n1.1 Pendahuluan\nKesehatan mental memliki pengertian keadaan dimana perkembangan fisik, intelektual dan emosional pada diri seseorang tumbuh, berkembang dan matang pada kehidupannya, menerima tanggung jawab, menemukan penyesuaian, memelihara aturan sosial dan tindakan dalam budayanya (Fakhriyani, 2019). Kesehatan mental berasal dari bahasa Inggris, yaitu mental hygiene. Mental berasal dari kata latin mens, mentis yang berarti jiwa, nyawa, sukma, roh, dan semangat. Sedangkan hygiene berasal dari kata yunani hygiene yang berarti ilmu tentang kesehatan (Kartikasari et al., 2022).'),
 Document(metadata={'bab': 'BAB I PRINSIP-PRINSIP DAN FUNGSI KESEHATAN MENTAL'}, page_content='Menurut Federasi Kesehatan Mental Dunia (World Federation for Mental Health), kesehatan mental adalah (1) kondisi yang memungkinkan adanya perkembangan yang optimal baik se

In [51]:
import pickle

# Save the docs object to a .pkl file
with open("docs3.pkl", "wb") as file:
    pickle.dump(docs, file)